# Time Series Prediction with BQML and AutoML

**Objectives**
 1. Learn how to use BQML to create a classification time-series model using `CREATE MODEL`.
 2. Learn how to use BQML to create a linear regression time-series model.
 3. Learn how to use AutoML Tables to build a time series model from data in BigQuery.

## Set up environment variables and load necessary libraries

In [5]:
PROJECT = "cloud-training-demos"  # Replace with your PROJECT.
REGION = "us-central1"            # Choose an available region for CAIP.

In [10]:
%env
PROJECT = PROJECT
REGION = REGION

In [ ]:
!pip freeze | grep google-cloud-bigquery==1.6.1 || pip install google-cloud-bigquery==1.6.1

## Review the dataset

In the previous lab we created the data we will use modeling and saved them as tables in BigQuery. Let's examine that table again to see that everything is as we expect. Then, we will build a model using BigQuery ML using this table.

In [11]:
%%bigquery --project $PROJECT
#standardSQL
SELECT
  *
FROM
  stock_market.percent_change_sp500
LIMIT
  10

,symbol,Date,Open,Close,tomorrow_close,tomo_close_m_close,close_MIN_prior_5_days,close_MIN_prior_20_days,close_MIN_prior_260_days,close_MAX_prior_5_days,...,close_STDDEV_prior_20_days,close_STDDEV_prior_260_days,close_values_prior_260,days_on_market,scaled_change,s_p_scaled_change,normalized_change,company,industry,direction
0,ALXN,2004-09-30,18.03,18.00,18.73,0.73,0.950556,0.874444,0.753889,1.004444,...,0.046965,0.150737,"[16.55, 17.5, 17.33, 17.67, 17.0, 16.9, 16.51,...",2155,0.040556,0.015181,0.025375,Alexion Pharmaceuticals Inc,Health Care,UP
1,APH,2007-05-25,34.76,34.74,34.96,0.22,0.994531,0.984744,0.945308,1.015256,...,0.017833,0.308593,"[56.08, 56.03, 55.88, 54.44, 53.81, 54.42, 54....",3909,0.006333,0.001570,0.004763,Amphenol Corp A,Information Technology,STAY
2,AVY,1993-03-29,28.37,28.75,29.00,0.25,0.956522,0.904348,0.830261,0.982609,...,0.022628,0.032504,"[27.0, 26.87, 26.5, 26.12, 26.37, 26.25, 26.37...",2164,0.008696,0.002662,0.006034,Avery Dennison Corp,Industrials,STAY
3,ADI,2003-12-05,48.55,47.16,46.60,-0.56,1.029474,0.951866,0.488550,1.055980,...,0.031501,0.149685,"[29.7, 31.27, 29.21, 31.2, 30.69, 30.84, 29.57...",4887,-0.011874,0.007348,-0.019223,Analog Devices Inc,Information Technology,DOWN
4,APA,1994-02-02,25.25,26.62,25.87,-0.75,0.896694,0.896694,0.783997,0.943651,...,0.022109,0.097928,"[20.87, 21.62, 21.37, 21.75, 21.37, 21.62, 21....",3059,-0.028174,-0.002676,-0.025498,Apache Corp,Energy,DOWN
5,A,2003-07-03,20.18,19.99,21.29,1.30,0.967984,0.937969,0.542771,1.015508,...,0.020946,0.153598,"[23.96, 24.16, 23.55, 23.2, 24.23, 23.65, 23.1...",909,0.065033,0.018992,0.046041,Agilent Technologies Inc,Health Care,UP
6,AFL,1992-06-26,27.11,26.88,27.45,0.57,0.995536,0.948289,0.693824,1.000000,...,0.018846,0.102334,"[19.78, 18.76, 19.44, 19.56, 19.21, 18.65, 18....",2008,0.021205,0.013608,0.007598,AFLAC Inc,Financials,STAY
7,ADM,1989-05-26,21.39,21.49,21.39,-0.10,0.991159,0.959516,0.646812,1.026989,...,0.016927,0.090423,"[16.48, 16.39, 16.58, 16.29, 16.19, 16.19, 16....",1554,-0.004653,-0.007898,0.003245,Archer-Daniels-Midland Co,Consumer Staples,STAY
8,ABT,1998-09-24,44.31,44.38,44.25,-0.13,0.958991,0.867508,0.819739,1.026589,...,0.044080,0.309961,"[62.0, 61.81, 63.44, 63.13, 63.75, 64.44, 64.8...",3912,-0.002929,0.001947,-0.004876,Abbott Laboratories,Health Care,STAY
9,ADSK,2007-07-10,44.89,44.90,45.38,0.48,1.001336,0.998441,0.669488,1.053229,...,0.021376,0.100488,"[34.26, 32.88, 32.86, 34.47, 34.46, 34.94, 34....",5550,0.010690,0.005721,0.004969,Autodesk Inc,Information Technology,STAY


## Using BQML

### Create classification model for `direction`

To create a model
1. Use `CREATE MODEL` and provide a destination table for resulting model. Alternatively we can use `CREATE OR REPLACE MODEL` which allows overwriting an existing model.
2. Use `OPTIONS` to specify the model type (linear_reg or logistic_reg). There are many more options [we could specify](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-create#model_option_list), such as regularization and learning rate, but we'll accept the defaults.
3. Provide the query which fetches the training data 

Have a look at [Step Two of this tutorial](https://cloud.google.com/bigquery/docs/bigqueryml-natality) to see another example.

**The query will take about two minutes to complete**


We'll start with creating a classification model to predict the `direction` of each stock. 

We'll take a random split using the `symbol` value. With about 500 different values, using `MOD(ABS(FARM_FINGERPRINT(symbol)), 15) = 1` will give 30 distinct `symbol` values which corresponds to about 171,000 training examples. After taking 70% for training, we will be building a model on about 110,000 training examples.

In [13]:
# TODO 1a
%%bigquery --project $PROJECT
#standardSQL
CREATE OR REPLACE MODEL
  stock_market.direction_model OPTIONS(model_type = "logistic_reg",
    input_label_cols = ["direction"]) AS
  -- query to fetch training data
SELECT
  symbol,
  Date,
  Open,
  Close,
  tomorrow_close,
  tomo_close_m_close,
  close_MIN_prior_5_days,
  close_MIN_prior_20_days,
  close_MIN_prior_260_days,
  close_MAX_prior_5_days,
  close_MAX_prior_20_days,
  close_MAX_prior_260_days,
  close_AVG_prior_5_days,
  close_AVG_prior_20_days,
  close_AVG_prior_260_days,
  close_STDDEV_prior_5_days,
  close_STDDEV_prior_20_days,
  close_STDDEV_prior_260_days,
  direction
FROM
  `stock_market.percent_change_sp500`
WHERE
  tomorrow_close IS NOT NULL
  AND MOD(ABS(FARM_FINGERPRINT(symbol)), 15) = 1
  AND MOD(ABS(FARM_FINGERPRINT(symbol)), 15*100) <= 15*70

""


## Get training statistics and examine training info

After creating our model, we can evaluate the performance using the [`ML.EVALUATE` function](https://cloud.google.com/bigquery-ml/docs/bigqueryml-natality#step_four_evaluate_your_model). With this command, we can find the precision, recall, accuracy F1-score and AUC of our classification model.

In [14]:
# TODO 1b
%%bigquery --project $PROJECT
#standardSQL
SELECT
  *
FROM
  ML.EVALUATE(MODEL `stock_market.direction_model`,
    (
    SELECT
      symbol,
      Date,
      Open,
      Close,
      tomorrow_close,
      tomo_close_m_close,
      close_MIN_prior_5_days,
      close_MIN_prior_20_days,
      close_MIN_prior_260_days,
      close_MAX_prior_5_days,
      close_MAX_prior_20_days,
      close_MAX_prior_260_days,
      close_AVG_prior_5_days,
      close_AVG_prior_20_days,
      close_AVG_prior_260_days,
      close_STDDEV_prior_5_days,
      close_STDDEV_prior_20_days,
      close_STDDEV_prior_260_days,
      direction
    FROM
      `stock_market.percent_change_sp500`
    WHERE
      tomorrow_close IS NOT NULL
      AND MOD(ABS(FARM_FINGERPRINT(symbol)), 15) = 1
      AND MOD(ABS(FARM_FINGERPRINT(symbol)), 15*100) > 15*70
      AND MOD(ABS(FARM_FINGERPRINT(symbol)), 15*100) <= 15*85))

,precision,recall,accuracy,f1_score,log_loss,roc_auc
0,0.495065,0.405386,0.563221,0.395176,1.079252,0.661035


We can also examine the training statistics collected by Big Query. To view training results we use the [`ML.TRAINING_INFO`](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-train) function.

In [16]:
#TODO 1c
%%bigquery --project $PROJECT
#standardSQL
SELECT
  *
FROM
  ML.TRAINING_INFO(MODEL `stock_market.direction_model`)
ORDER BY iteration

,training_run,iteration,loss,eval_loss,learning_rate,duration_ms
0,0,0,0.343544,0.348294,0.2,16518
1,0,1,0.315221,0.328672,0.4,20561
2,0,2,0.290389,0.318685,0.8,21722


### Compare to simple benchmark

Another way to asses the performance of our model is to compare with a simple benchmark. We can do this by seeing what kind of accuracy we would get using the naive strategy of just predicted the majority class. For the training dataset, the majority class is 'STAY'. The following query we can see how this naive strategy would perform on the eval set.

In [17]:
%%bigquery --project $PROJECT
#standardSQL
WITH
  eval_data AS (
  SELECT
    symbol,
    Date,
    Open,
    Close,
    tomorrow_close,
    tomo_close_m_close,
    close_MIN_prior_5_days,
    close_MIN_prior_20_days,
    close_MIN_prior_260_days,
    close_MAX_prior_5_days,
    close_MAX_prior_20_days,
    close_MAX_prior_260_days,
    close_AVG_prior_5_days,
    close_AVG_prior_20_days,
    close_AVG_prior_260_days,
    close_STDDEV_prior_5_days,
    close_STDDEV_prior_20_days,
    close_STDDEV_prior_260_days,
    direction
  FROM
    `stock_market.percent_change_sp500`
  WHERE
    tomorrow_close IS NOT NULL
    AND MOD(ABS(FARM_FINGERPRINT(symbol)), 15) = 1
    AND MOD(ABS(FARM_FINGERPRINT(symbol)), 15*100) > 15*70
    AND MOD(ABS(FARM_FINGERPRINT(symbol)), 15*100) <= 15*85)
SELECT
  direction,
  (COUNT(direction)* 100 / (
    SELECT
      COUNT(*)
    FROM
      eval_data)) AS percentage
FROM
  eval_data
GROUP BY
  direction

,direction,percentage
0,STAY,55.094426
1,UP,22.798872
2,DOWN,22.106702


So, the naive strategy of just guessing the majority class would have accuracy of 0.5509 on the eval dataset, just below our BQML model. 

### Create regression model for `normalized change`

We can also use BigQuery to train a regression model to predict the normalized change for each stock. To do this in BigQuery we need only change the OPTIONS when calling `CREATE OR REPLACE MODEL`. This will give us a more precise prediction rather than just predicting if the stock will go up, down, or stay the same. Thus, we can treat this problem as either a regression problem or a classification problem, depending on the business needs.


In [ ]:
#TODO 2a
%%bigquery --project $PROJECT
#standardSQL
CREATE OR REPLACE MODEL
  stock_market.price_model OPTIONS(model_type = "linear_reg",
    input_label_cols = ["normalized_change"]) AS
  -- query to fetch training data
SELECT
  symbol,
  Date,
  Open,
  Close,
  tomorrow_close,
  tomo_close_m_close,
  close_MIN_prior_5_days,
  close_MIN_prior_20_days,
  close_MIN_prior_260_days,
  close_MAX_prior_5_days,
  close_MAX_prior_20_days,
  close_MAX_prior_260_days,
  close_AVG_prior_5_days,
  close_AVG_prior_20_days,
  close_AVG_prior_260_days,
  close_STDDEV_prior_5_days,
  close_STDDEV_prior_20_days,
  close_STDDEV_prior_260_days,
  normalized_change
FROM
  `stock_market.percent_change_sp500`
WHERE
  normalized_change IS NOT NULL
  AND MOD(ABS(FARM_FINGERPRINT(symbol)), 15) = 1
  AND MOD(ABS(FARM_FINGERPRINT(symbol)), 15*100) <= 15*70

Just as before we can examine the evaluation metrics for our regression model and examine the training statistics in Big Query

In [ ]:
%%bigquery --project $PROJECT
#standardSQL
SELECT
  *
FROM
  ML.EVALUATE(MODEL `stock_market.price_model`,
    (
    SELECT
      symbol,
      Date,
      Open,
      Close,
      tomorrow_close,
      tomo_close_m_close,
      close_MIN_prior_5_days,
      close_MIN_prior_20_days,
      close_MIN_prior_260_days,
      close_MAX_prior_5_days,
      close_MAX_prior_20_days,
      close_MAX_prior_260_days,
      close_AVG_prior_5_days,
      close_AVG_prior_20_days,
      close_AVG_prior_260_days,
      close_STDDEV_prior_5_days,
      close_STDDEV_prior_20_days,
      close_STDDEV_prior_260_days,
      normalized_change
    FROM
      `stock_market.percent_change_sp500`
    WHERE
      normalized_change IS NOT NULL
      AND MOD(ABS(FARM_FINGERPRINT(symbol)), 15) = 1
      AND MOD(ABS(FARM_FINGERPRINT(symbol)), 15*100) > 15*70
      AND MOD(ABS(FARM_FINGERPRINT(symbol)), 15*100) <= 15*85))

In [ ]:
%%bigquery --project $PROJECT
#standardSQL
SELECT
  *
FROM
  ML.TRAINING_INFO(MODEL `stock_market.price_model`)
ORDER BY iteration

## Train a Time Series model using AutoML Tables

### Step 1. Launch AutoML

#TODO 3

Within the GCP console, navigate to Tables in the console menu.

<img src='assets/console_menu_tables.png' width='50%'>

### Step 2. Create a Dataset

Select New Dataset and give it a name like `stock_market`. In the section on Importing data, select the option to import your data from a BigQuery Table. Fill in the details for your project, the dataset ID, and the table ID.

<img src='assets/import_data_options.png' width='50%'>

### Step 3. Import the Data

Once you have created the dataset you can then import the data. This will take a few minutes.

<img src='assets/importing_data.png' width='50%'>

### Step 4. Train the model.

Once the data has been imported into the dataset. You can examine the Schema of your data, Analyze the properties and values of the features and ultimately Train the model. Here you can also determine the label column and features for training the model. Since we are doing a classifcation model, we'll use `direction` as our target column.

<img src='assets/schema_analyze_train.png' width='80%'>

Under the `Train` tab you can choose the features to use when training. Select the same features as we used above. 

<img src='assets/train_model.png' width='50%'>

### Step 5. Evaluate your model.

Training can take many hours. But once training is complete you can inspect the evaluation metrics of your model. Since this is a classification task, we can also adjust the threshold and explore how different thresholds will affect your evaluation metrics. Also on that page, we can explore the feature importance of the various features used in the model and view confusion matrix for our model predictions.

<img src='assets/eval_metrics.png' width='80%'>

### Step 6. Predict with the trained model. 

Once the model is done training, navigate to the Models page and Deploy the model, so we can test prediction. 

<img src='assets/deploy_model.png' width='80%'>

When calling predictions, you can call batch prediction jobs by specifying a BigQuery table or csv file. Or you can do online prediction for a single instance.

Copyright 2019 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License